In [1]:
import numpy as np
import jsonlines
from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
import time

Using TensorFlow backend.


### 這裡負責label喔

keys = ['appeared', 'exports',  'label', 'histogram', 'imports', 'general', 'sha256', 'section', 'byteentropy', 'header']
keys_cnn = ['strings','sha256', 'appeared', 'histogram', 'byteentropy', 'general']#[9,1,1,256,256,10]
keys_rnn = ['exports', 'imports', 'section', 'header']

keys_strings = ['avlength', 'numstrings', 'registry', 'urls', 'MZ', 'printables', 'entropy', 'paths', 'printabledist']
keys_general = ['exports', 'has_resources', 'imports', 'symbols', 'has_signature', 'has_relocations', 'has_debug', 'vsize', 'size', 'has_tls']

In [4]:
#save all label

for name in range(0,7):
    if name < 6:
        path = "./ember/train_features_"+str(name)+".jsonl"
    else:
        path = "./ember/test_features.jsonl"
    origin_feature = []
    with jsonlines.open(path) as reader:
        for obj in reader:
            origin_feature.append(obj)
    n = len(origin_feature)
    print(n)
    keys = ['appeared', 'exports',  'label', 'histogram', 'imports', 'general', 'sha256', 'section', 'byteentropy', 'header']
    keys_cnn = ['strings','sha256', 'appeared', 'histogram', 'byteentropy', 'general']#[9,1,1,256,256,10]
    keys_rnn = ['exports', 'imports', 'section', 'header']
    
    keys_strings = ['avlength', 'numstrings', 'registry', 'urls', 'MZ', 'printables', 'entropy', 'paths', 'printabledist']
    keys_general = ['exports', 'has_resources', 'imports', 'symbols', 'has_signature', 'has_relocations', 'has_debug', 'vsize', 'size', 'has_tls']
    label = []
    for i in range(n):
        label.append(origin_feature[i]['label'])
    np.save("./label/"+ str(name) +".npy",label)

50000
170000
170000
170000
170000
170000
200000


#####################################################################

### 這裡負責字串

In [5]:
qq = [0,1,2,3,4,5,6]
#qq = [6]


for num in qq:

    print("Data",num,"\n")
    path = "./ember/train_features_"+ str(num) +".jsonl"
    
    if num == 6:
        path = "./ember/test_features.jsonl"
    
    print("Read Data...\n")
    origin_feature = []
    with jsonlines.open(path) as reader:
        for obj in reader:
            origin_feature.append(obj)
    n = len(origin_feature)
    
    print("Read strings...\n")
    # imports
    
    rnn_feature = []
    for i in range(n):
        rnn_feature.append([])
    
    for i in range(n):
        tmp = origin_feature[i]['imports'].keys()
        for key in tmp:
            tmp_n = len(origin_feature[i]['imports'][key])
            for j in range(tmp_n):
                rnn_feature[i].append(origin_feature[i]['imports'][key][j])
    '''
    # header coff
    for i in range(n):
        rnn_feature[i].append(origin_feature[i]['header']['coff']['machine'])
        tmp_n = len(origin_feature[i]['header']['coff']['characteristics'])
        for j in range(tmp_n):
            rnn_feature[i].append(origin_feature[i]['header']['coff']['characteristics'][j])
    '''

    # exports
    for i in range(n):
        tmp_n = len(origin_feature[i]['exports'])
        if tmp_n != 0:
            for obj in origin_feature[i]['exports']:
                rnn_feature[i].append(obj)
    save = []
    for i in range(n):
        htable = np.zeros(256)
        for obj in rnn_feature[i]:
            h = hash(obj) % 256
            htable[h] += 1
        save.append(htable)
    save = np.array(save)
    
    print("Save data...\n")
    # 存檔
    print(save.shape)
    np.save("./feature/rnn_feature" + str(num) + ".npy", save)
    #np.save("./feature/rnn_test_feature.npy", save)


Data 0 

Read Data...

Read strings...

Save data...

(50000, 256)
Data 1 

Read Data...

Read strings...

Save data...

(170000, 256)
Data 2 

Read Data...

Read strings...

Save data...

(170000, 256)
Data 3 

Read Data...

Read strings...

Save data...

(170000, 256)
Data 4 

Read Data...

Read strings...

Save data...

(170000, 256)
Data 5 

Read Data...

Read strings...

Save data...

(170000, 256)
Data 6 

Read Data...

Read strings...

Save data...

(200000, 256)


In [4]:
start = time.time()
w2v_model = Word2Vec(rnn_feature,min_count = 4, size=200, sg = 1)
w2v_model.save("large_w2v.model")
end = time.time()
spend = end - start
print(spend)

3858.5670750141144


### 這裡負責數字

In [3]:
keys_strings = ['avlength', 'numstrings', 'registry', 'urls', 'MZ', 'printables', 'entropy', 'paths', 'printabledist']
keys_general = ['exports', 'has_resources', 'imports', 'symbols', 'has_signature', 'has_relocations', 'has_debug', 'vsize', 'size', 'has_tls']

for num in range(7):
    
    print(num)
    
    path = "./ember/train_features_"+ str(num) +".jsonl"
    
    if num == 6:
        path = "./ember/test_features.jsonl"
        
    origin_feature = []
    with jsonlines.open(path) as reader:
        for obj in reader:
            origin_feature.append(obj)
    n = len(origin_feature)
    print(n)
    output = []
    for obj in range(n):
        feature_num = []
        for i in range(4):
            feature_num.append([])
        for j in keys_strings:
            if j != 'printabledist':
                feature_num[0].append(origin_feature[obj]['strings'][j])
            else:
                for k in range(96):
                    feature_num[0].append(origin_feature[obj]['strings'][j][k])
        for i in range(256):
            feature_num[1].append(origin_feature[obj]['histogram'][i])
        for i in range(256):
            feature_num[2].append(origin_feature[obj]['byteentropy'][i])
        for i in keys_general:
            feature_num[3].append(origin_feature[obj]['general'][i])
        for i in range(256 - 104):
            feature_num[0].append(0)
        for i in range(256 - 10):
            feature_num[3].append(0)
        output.append(feature_num)
    tmp = np.array(output)
    np.save("./feature/num_feature"+str(num)+".npy",tmp)

6
200000


### 這裡把string還有number串接起來

In [6]:
for num in range(7):
    if num != 6:
        print("Import npy...\n")
        tmp = np.load("./feature/num_feature"+str(num)+".npy")
        save = np.load("./feature/rnn_feature"+str(num)+".npy")
        save_n = save.shape[0]
        print("Deal Data...\n")
        save = save.reshape(save_n, 1, 256)
        all_feature = np.hstack((tmp,save))
        print("Save Data...\n")
        np.save("./feature/all_features"+str(num)+".npy",all_feature)
    else:
        print("test")
        # test data
        tmp = np.load("./feature/num_feature6.npy")
        save = np.load("./feature/rnn_feature6.npy")
        save_n = save.shape[0]
        save = save.reshape(save_n, 1, 256)
        all_feature = np.hstack((tmp,save))
        np.save("./feature/all_test.npy",all_feature)

Import npy...

Deal Data...

Save Data...

Import npy...

Deal Data...

Save Data...

Import npy...

Deal Data...

Save Data...

Import npy...

Deal Data...

Save Data...

Import npy...

Deal Data...

Save Data...

Import npy...

Deal Data...

Save Data...

test
